# 03 Model Improvement and Time-Based Validation (Telecom Churn / Cease)

This notebook trains **leakage-safe churn models** for the business objective:

> **Prioritise retention resources by calling customers most likely to place a cease in the next 30 days.**

## Objectives
- Load feature dataset from Notebook 02
- Use a **time-based train / validation / test split**
- Train baseline models (**Logistic Regression** + **Random Forest**)
- Evaluate with classification and business metrics
- Optimise thresholds for **Top K% retention capacity**
- Create **gains / lift** charts
- Explain drivers (feature importance + SHAP if available)
- Score external/future snapshot datasets


In [ ]:
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import (
    roc_auc_score, average_precision_score, classification_report,
    precision_recall_curve
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 200)


In [ ]:
# Paths
cwd = Path.cwd()
repo_dir = cwd.parent if cwd.name.lower() in {"notebook", "notebooks"} else cwd

features_path_default = repo_dir / "outputs" / "features" / "telecom_churn_model_features_v1.parquet"
features_path_csv = repo_dir / "outputs" / "features" / "telecom_churn_model_features_v1.csv"
features_path_sample = Path("/mnt/data/telecom_churn_model_features_v1.parquet")  # optional if copied here manually

if features_path_default.exists():
    features_path = features_path_default
elif features_path_csv.exists():
    features_path = features_path_csv
elif features_path_sample.exists():
    features_path = features_path_sample
else:
    raise FileNotFoundError("Feature dataset not found. Run Notebook 02 first.")

print("Loading features from:", features_path)
if features_path.suffix.lower() == ".parquet":
    df = pd.read_parquet(features_path)
else:
    df = pd.read_csv(features_path)

print(df.shape)
display(df.head(3))


In [ ]:
# Basic data prep and sanitisation
df["snapshot_date"] = pd.to_datetime(df["snapshot_date"], errors="coerce")
df = df.dropna(subset=["snapshot_date"]).copy()

# Enforce target type
df["target_cease_30d"] = pd.to_numeric(df["target_cease_30d"], errors="coerce").fillna(0).astype(int)

# Optional: drop highly leaky or identifier columns from modelling features
id_cols = ["unique_customer_identifier", "snapshot_date"]
target_col = "target_cease_30d"

# Remove extreme hold ratios if present (cap to reduce outlier distortion)
for c in ["avg_hold_ratio_30d", "max_hold_ratio_30d"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")
        upper = df[c].quantile(0.99)
        df[c] = df[c].clip(upper=upper)

# Convert obvious numeric columns safely
for c in df.columns:
    if c in id_cols + [target_col]:
        continue
    # leave objects/categoricals as-is; convert only if likely numeric but stored as text
    if df[c].dtype == "object":
        # keep as categorical if too many non-numeric values
        converted = pd.to_numeric(df[c], errors="coerce")
        # if at least 80% values become numeric, treat as numeric
        if converted.notna().mean() >= 0.8:
            df[c] = converted

print("Date range:", df["snapshot_date"].min(), "to", df["snapshot_date"].max())
print("Target rate:", round(df[target_col].mean()*100, 2), "%")


## 1) Time-based train / validation / test split

We split by **snapshot_date** (not random split) to match real deployment.

- **Train**: oldest period
- **Validation**: middle period
- **Test**: most recent period


In [ ]:
# Time split by quantiles (works on sample and full data)
q1 = df["snapshot_date"].quantile(0.70)
q2 = df["snapshot_date"].quantile(0.85)

train_df = df[df["snapshot_date"] <= q1].copy()
valid_df = df[(df["snapshot_date"] > q1) & (df["snapshot_date"] <= q2)].copy()
test_df  = df[df["snapshot_date"] > q2].copy()

print("Train:", train_df.shape, train_df["snapshot_date"].min(), "->", train_df["snapshot_date"].max())
print("Valid:", valid_df.shape, valid_df["snapshot_date"].min(), "->", valid_df["snapshot_date"].max())
print("Test :", test_df.shape,  test_df["snapshot_date"].min(), "->", test_df["snapshot_date"].max())

display(pd.DataFrame({
    "split": ["train","valid","test"],
    "rows": [len(train_df), len(valid_df), len(test_df)],
    "target_rate_pct": [train_df[target_col].mean()*100, valid_df[target_col].mean()*100, test_df[target_col].mean()*100]
}))


In [ ]:
# Features used for modelling
drop_cols = id_cols + [target_col]

X_train = train_df.drop(columns=drop_cols, errors="ignore")
y_train = train_df[target_col]

X_valid = valid_df.drop(columns=drop_cols, errors="ignore")
y_valid = valid_df[target_col]

X_test = test_df.drop(columns=drop_cols, errors="ignore")
y_test = test_df[target_col]

# Identify numeric vs categorical columns
numeric_cols = X_train.select_dtypes(include=["number", "bool"]).columns.tolist()
categorical_cols = [c for c in X_train.columns if c not in numeric_cols]

print("Numeric cols:", len(numeric_cols))
print("Categorical cols:", len(categorical_cols))
print("Sample categorical:", categorical_cols[:10])


In [ ]:
# Preprocessing pipeline
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_cols),
        ("cat", categorical_transformer, categorical_cols)
    ],
    remainder="drop"
)


## 2) Train baseline models

We start with:
- **Logistic Regression** (interpretable baseline)
- **Random Forest** (non-linear baseline)


In [ ]:
# Baseline models
logreg_pipe = Pipeline(steps=[
    ("prep", preprocess),
    ("model", LogisticRegression(max_iter=1000, class_weight="balanced", n_jobs=None))
])

rf_pipe = Pipeline(steps=[
    ("prep", preprocess),
    ("model", RandomForestClassifier(
        n_estimators=300,
        max_depth=None,
        min_samples_leaf=5,
        random_state=42,
        n_jobs=-1,
        class_weight="balanced_subsample"
    ))
])

logreg_pipe.fit(X_train, y_train)
rf_pipe.fit(X_train, y_train)

# Validation predictions
valid_scores_lr = logreg_pipe.predict_proba(X_valid)[:, 1]
valid_scores_rf = rf_pipe.predict_proba(X_valid)[:, 1]

def metric_summary(y_true, y_score, model_name):
    return {
        "model": model_name,
        "roc_auc": roc_auc_score(y_true, y_score) if len(np.unique(y_true)) > 1 else np.nan,
        "pr_auc": average_precision_score(y_true, y_score) if len(np.unique(y_true)) > 1 else np.nan,
        "base_rate": float(np.mean(y_true))
    }

metrics_df = pd.DataFrame([
    metric_summary(y_valid, valid_scores_lr, "LogisticRegression"),
    metric_summary(y_valid, valid_scores_rf, "RandomForest")
])
display(metrics_df)


In [ ]:
# Pick best baseline by validation ROC-AUC (fallback to RF on tie/nan)
if metrics_df["roc_auc"].notna().any():
    best_name = metrics_df.sort_values("roc_auc", ascending=False)["model"].iloc[0]
else:
    best_name = "RandomForest"

best_model = logreg_pipe if best_name == "LogisticRegression" else rf_pipe
print("Selected model:", best_name)


## 3) Threshold optimisation for retention capacity (Top K%)

The business can only call a **limited proportion** of customers.  
So we optimise on **Top K% prioritisation** rather than a fixed 0.5 threshold.


In [ ]:
# Top-K evaluation helper
def topk_metrics(y_true, scores, k_frac=0.1):
    n = len(scores)
    k = max(1, int(np.floor(n * k_frac)))
    order = np.argsort(-scores)
    top_idx = order[:k]

    y_true = np.asarray(y_true)
    top_y = y_true[top_idx]

    captured = top_y.sum()
    total_positives = y_true.sum()
    recall_at_k = captured / total_positives if total_positives > 0 else np.nan
    precision_at_k = top_y.mean() if len(top_y) > 0 else np.nan
    lift_at_k = (precision_at_k / y_true.mean()) if y_true.mean() > 0 else np.nan
    threshold = scores[order[k-1]] if k > 0 else 1.0

    return {
        "k_frac": k_frac,
        "k_customers": k,
        "threshold": float(threshold),
        "precision_at_k": float(precision_at_k),
        "recall_at_k": float(recall_at_k),
        "lift_at_k": float(lift_at_k)
    }

# Evaluate multiple capacity levels on validation
valid_scores_best = best_model.predict_proba(X_valid)[:, 1]
topk_grid = [0.01, 0.02, 0.05, 0.10, 0.15, 0.20, 0.30]
topk_valid_df = pd.DataFrame([topk_metrics(y_valid.values, valid_scores_best, k) for k in topk_grid])
display(topk_valid_df)


In [ ]:
# Choose operational capacity (edit based on business capacity)
RETENTION_TOP_K = 0.10  # e.g., call top 10% highest-risk customers

chosen = topk_valid_df.loc[(topk_valid_df["k_frac"] - RETENTION_TOP_K).abs().idxmin()].to_dict()
score_threshold = chosen["threshold"]

print(f"Chosen Top K% capacity: {chosen['k_frac']*100:.0f}%")
print(f"Validation score threshold: {score_threshold:.4f}")
print(f"Precision@K: {chosen['precision_at_k']:.3f} | Recall@K: {chosen['recall_at_k']:.3f} | Lift@K: {chosen['lift_at_k']:.2f}")


## 4) Final evaluation on the test set (most recent period)


In [ ]:
# Score test set
test_scores = best_model.predict_proba(X_test)[:, 1]
test_pred_topk = (test_scores >= score_threshold).astype(int)

# Standard metrics
test_metrics = pd.DataFrame([metric_summary(y_test, test_scores, f"{best_name} (test)")])
display(test_metrics)

# Capacity-based metric at chosen threshold
test_topk = topk_metrics(y_test.values, test_scores, RETENTION_TOP_K)
display(pd.DataFrame([test_topk]))

print("\nClassification report using Top-K-derived threshold:")
print(classification_report(y_test, test_pred_topk, digits=3))


## 5) Business-ready gains and lift charts

These charts show how well the model helps the business prioritise calls.


In [ ]:
def gains_lift_table(y_true, scores, n_bins=10):
    dfp = pd.DataFrame({"y": np.asarray(y_true), "score": np.asarray(scores)}).sort_values("score", ascending=False).reset_index(drop=True)
    dfp["decile"] = pd.qcut(dfp.index + 1, q=n_bins, labels=False) + 1  # 1=highest score bucket
    # convert so decile 1 is top bucket already
    grp = dfp.groupby("decile", as_index=False).agg(
        customers=("y","size"),
        churners=("y","sum"),
        avg_score=("score","mean")
    )
    grp["cum_customers"] = grp["customers"].cumsum()
    grp["cum_churners"] = grp["churners"].cumsum()
    total_customers = grp["customers"].sum()
    total_churners = grp["churners"].sum()
    base_rate = total_churners / total_customers if total_customers > 0 else np.nan
    grp["cum_customer_pct"] = grp["cum_customers"] / total_customers
    grp["cum_churn_capture_pct"] = grp["cum_churners"] / total_churners if total_churners > 0 else np.nan
    grp["precision"] = grp["churners"] / grp["customers"]
    grp["lift"] = grp["precision"] / base_rate if base_rate > 0 else np.nan
    return grp

gl_valid = gains_lift_table(y_valid, valid_scores_best, n_bins=10)
gl_test = gains_lift_table(y_test, test_scores, n_bins=10)

display(gl_test)


In [ ]:
# Plot gains chart (test)
fig, ax = plt.subplots(figsize=(7,5))
ax.plot([0,1], [0,1], linestyle="--")
ax.plot(gl_test["cum_customer_pct"], gl_test["cum_churn_capture_pct"], marker="o")
ax.set_title("Gains Chart (Test)")
ax.set_xlabel("Cumulative % Customers Contacted")
ax.set_ylabel("Cumulative % Churners Captured")
plt.tight_layout()
plt.show()

# Plot lift by decile (test)
fig, ax = plt.subplots(figsize=(7,5))
ax.bar(gl_test["decile"].astype(str), gl_test["lift"])
ax.set_title("Lift by Decile (Test)")
ax.set_xlabel("Decile (1 = highest risk)")
ax.set_ylabel("Lift vs Average")
plt.tight_layout()
plt.show()


## 6) Explainability: feature importance (and SHAP if available)


In [ ]:
# Feature names after preprocessing
prep_fitted = best_model.named_steps["prep"]
feature_names = prep_fitted.get_feature_names_out()

model_obj = best_model.named_steps["model"]

# Model-specific feature importance
if hasattr(model_obj, "feature_importances_"):
    importances = model_obj.feature_importances_
    imp_df = pd.DataFrame({"feature": feature_names, "importance": importances}).sort_values("importance", ascending=False)
elif hasattr(model_obj, "coef_"):
    coefs = model_obj.coef_[0]
    imp_df = pd.DataFrame({"feature": feature_names, "importance": np.abs(coefs), "coef": coefs}).sort_values("importance", ascending=False)
else:
    imp_df = pd.DataFrame(columns=["feature","importance"])

display(imp_df.head(30))

# Plot top features
topn = 20
fig, ax = plt.subplots(figsize=(9,6))
top = imp_df.head(topn).sort_values("importance", ascending=True)
ax.barh(top["feature"], top["importance"])
ax.set_title(f"Top {topn} Model Drivers ({best_name})")
ax.set_xlabel("Importance")
plt.tight_layout()
plt.show()


In [ ]:
# Optional SHAP (if installed). Safe to skip if not available.
try:
    import shap
    # Sample a subset for speed
    X_valid_sample = X_valid.sample(min(500, len(X_valid)), random_state=42)
    X_valid_tx = prep_fitted.transform(X_valid_sample)

    if hasattr(model_obj, "feature_importances_"):  # tree model
        explainer = shap.TreeExplainer(model_obj)
        shap_values = explainer.shap_values(X_valid_tx)
        # Binary classifier shap can be list or array depending on version
        sv = shap_values[1] if isinstance(shap_values, list) else shap_values
        shap.summary_plot(sv, X_valid_tx, feature_names=feature_names, show=True)
    else:
        print("SHAP tree summary skipped (selected model is not tree-based).")
except Exception as e:
    print("SHAP not available or skipped:", e)


## 7) Score external / future snapshot datasets

Use the same **feature engineering logic from Notebook 02** to build a feature table for new snapshots, then score with the trained model.

The output should include:
- `unique_customer_identifier`
- `snapshot_date`
- `churn_score`
- `priority_rank`
- `priority_band` (High / Medium / Low)


In [ ]:
# Example scoring output on test split (acts like external scoring format)
scored = test_df[["unique_customer_identifier", "snapshot_date", "target_cease_30d"]].copy()
scored["churn_score"] = test_scores
scored = scored.sort_values("churn_score", ascending=False).reset_index(drop=True)
scored["priority_rank"] = np.arange(1, len(scored) + 1)
scored["priority_pct"] = scored["priority_rank"] / len(scored)

# Example priority bands based on rank
scored["priority_band"] = np.select(
    [scored["priority_pct"] <= 0.10, scored["priority_pct"] <= 0.30],
    ["High", "Medium"],
    default="Low"
)

display(scored.head(20))


In [ ]:
# Save outputs for business / ops handoff
out_dir = repo_dir / "outputs" / "model_outputs"
out_dir.mkdir(parents=True, exist_ok=True)

# Save metrics and scored outputs
metrics_df.to_csv(out_dir / "validation_model_metrics.csv", index=False)
topk_valid_df.to_csv(out_dir / "validation_topk_metrics.csv", index=False)
test_metrics.to_csv(out_dir / "test_model_metrics.csv", index=False)
gl_test.to_csv(out_dir / "test_gains_lift_table.csv", index=False)
scored.to_csv(out_dir / "retention_prioritisation_scores_test_example.csv", index=False)

print("Saved to:", out_dir)


## Business interpretation guide (use in your presentation)

- **Model score** ranks customers by likelihood to cease in next 30 days.
- **Top K% threshold** maps directly to call-centre capacity.
- **Recall@K** shows what % of likely churners you capture when only calling top K%.
- **Lift** shows how much better the prioritised list performs vs calling customers at random.

This is exactly what the business asked for: **better prioritisation of retention resources**.
